# FIT5149 Applied Data Analysis
## Assignment 2: Sentiment Classification of YELP Product Reviews

### Group Details:

#### Group Number : 69
#### Team Members : 
1. Rishabh Kochhar - 29869560
2. Subhasish Sarkar - 29819253
3. Ajay Ganapathy - 29822270

###  Importing Required Libraries

In [186]:
#!pip install mlxtend
nltk.download('all')
from __future__ import division
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import RegexpTokenizer
%matplotlib inline 
import nltk
from nltk.corpus import reuters
from nltk.corpus import stopwords 
from nltk.util import ngrams
from nltk.tokenize import MWETokenizer
from nltk.stem import SnowballStemmer
from itertools import chain
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from  sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression as log_r
from sklearn.naive_bayes import MultinomialNB as mnb
from sklearn.ensemble import VotingClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |

[nltk_data]    |   Package ppattach is already up-to-date!
[nltk_data]    | Downloading package problem_reports to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package problem_reports is already up-to-date!
[nltk_data]    | Downloading package propbank to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package propbank is already up-to-date!
[nltk_data]    | Downloading package ptb to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package ptb is already up-to-date!
[nltk_data]    | Downloading package product_reviews_1 to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package product_reviews_1 is already up-to-date!
[nltk_data]    | Downloading package product_reviews_2 to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package product_reviews_2 is already up-to-date!
[nltk_data]    | Down

[nltk_data]    |   Package snowball_data is already up-to-date!
[nltk_data]    | Downloading package bllip_wsj_no_aux to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package bllip_wsj_no_aux is already up-to-date!
[nltk_data]    | Downloading package word2vec_sample to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package word2vec_sample is already up-to-date!
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package panlex_swadesh is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger

In [187]:
#reading the labeled dataset
df=pd.read_csv('labeled_data.csv')

In [188]:
# Checking the top few rows of the dataframe
df.head()

,text,label
0,The new rule is - \r\nif you are waiting for a...,4
1,"Flirted with giving this two stars, but that's...",3
2,I was staying at planet Hollywood across the s...,5
3,Food is good but prices are super expensive. ...,2
4,Worse company to deal with they do horrible wo...,1


## Pre-processing labeled data and feature extraction

In this step, we clean the raw text by performing several tasks as mentioned below:
1. Normalizing the raw text
2. Handling Contractions
3. Removing punctuations
4. Number Encoding
5. Removing spaces, newline characters
6. Tokenization
7. Certain stopwords removal
8. Processing Bigrams and Trigrams

First, we preprocess the labeled dataset, extract features in the labeled dataset and build models

In [189]:
# Make a list of relevant POS tags since we do not care about the rest
POS_TAGS=['CC','JJ','JJR','JJS','RBR','RBS','MD','PDT','VB','VBD','VBG','VBN',\
          'VBP','VBZ','WRB','WP','WDT'] 

In [190]:
#function to clean the raw text where we convert the raw text to lower case, 
#expand contractions such as can't to cannot, remove punctuations, newline characters,
#tabs, tokenize the raw text
def CleanData(row_id,raw_text):
    cleaned_text = list()
    
    # Lower case
    proc_text=raw_text.lower()
    
    # Handling Contractions
    # specific
    proc_text = re.sub(r"won\'t", "will not", proc_text)
    proc_text = re.sub(r"can\'t", "can not", proc_text)
    # general
    proc_text = re.sub(r"n\'t", " not", proc_text)
    proc_text = re.sub(r"\'re", " are", proc_text)
    proc_text = re.sub(r"\'s", " is", proc_text)
    proc_text = re.sub(r"\'d", " would", proc_text)
    proc_text = re.sub(r"\'ll", " will", proc_text)
    proc_text = re.sub(r"\'t", " not", proc_text)
    proc_text = re.sub(r"\'ve", " have", proc_text)
    proc_text = re.sub(r"\'m", " am", proc_text)
    # Handling punctuations
    proc_text=re.sub(r'[^a-z\s0-9]',' ',proc_text)
    # Replacing all numbers with custom string
    proc_text=re.sub(r'\d+','NUM',proc_text)
    # Handling multiple spaces
    proc_text=re.sub(r'\s+', ' ',proc_text, flags=re.I)
    # Handling tabs and newline charecters
    proc_text=re.sub(r'\r|\n|\t','',proc_text)
    # Tokenizing
    tokens = RegexpTokenizer(r"[\w]{2,}").tokenize(proc_text)
    # Appending to new list
    cleaned_text=tokens
    return (row_id,cleaned_text)

In [191]:
#removing unwanted characters such as \r\n\t and preserving the emojis
df['emojis'] = [None]*df.shape[0]
for row_id, row in df.iterrows():
    text = row.text
    text = re.sub(r'\r|\n|\t','',text)
    emojis = re.findall(r'(:\)|:\(|:\-\(|:\-\)|:\*|:\-\*|:\/|:\-\/|;\)|:D)',text)
    text = re.sub(r'[^A-Za-z0-9]',' ',text)
    text = re.sub(r'\s+',' ',text)
    text = text.split()
    df['num_emojis'] = len(emojis)

In [192]:
#function to calculate the word density or average word length
def avgWordLength(text):
    words = text.split()
    word_lengths = [len(word) for word in words]
    return np.mean(word_lengths)

Here, we extract several feature based numerical columns such as word counts, character counts, word density, punctuation counts, total length, capital word count, exclamation and question mark count, unique words count, etc.

In [193]:
#function to extract quantitative features
punc = ['.',',','/',':',';','-','(',')']

def extract_features(df) :
    df['word_count'] = df['text'].apply(lambda x : len(x.split()))
    df['char_count'] = df['text'].apply(lambda x : len(x.replace(" ","")))
    df['word_density'] = df['text'].apply(avgWordLength)
    df['punc_count'] = df['text'].apply(lambda x : len([a for a in x if a in punc]))
    df['total_length'] = df['text'].apply(len)
    df['capitals'] = df['text'].apply(lambda comment: sum(1 for c in comment.split() if len(c)>1 and c.isupper()))
    df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/float(row['word_count']),axis=1)
    df['num_exclamation_marks'] =df['text'].apply(lambda x: x.count('!'))
    df['num_question_marks'] = df['text'].apply(lambda x: x.count('?'))
    df['num_unique_words'] = df['text'].apply(lambda x: len(set(w for w in x.split())))
    df['words_vs_unique'] = df['num_unique_words'] / df['word_count']
    print('Done')
    return df

In [194]:
# run the feature extraction function, and store the new columns generated into our original dataframe
df=extract_features(df)

Done


In [195]:
# Make a list of custom stop words 
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))

to_keep = ['against', "aren't", 'couldn', "couldn't", 'didn', "didn't",\
                 'doesn', "doesn't", 'don', "don't", 'few', "further", 'hadn', "hadn't", \
                 'has', 'hasn', "hasn't", "haven", "haven't", 'isn', "isn't", 'might',\
                 "mightn't", 'mustn', "mustn't", "needn't", "not", "no", "nor", "off",\
                  "shan't", "should", "should've", "shouldn't",\
                 "some", "such", "under", "until", "very", "wasn't", "weren't","wouldn't",\
                 'ain', 'aren', 'wasn', 'weren', "weren't", 'won', "won't", 'wouldn']

stop_words = [words for words in stop_words if not words in to_keep]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rkoc0004\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [196]:
# taking a look into the new columns generated
df.columns

Index(['text', 'label', 'emojis', 'num_emojis', 'word_count', 'char_count',
       'word_density', 'punc_count', 'total_length', 'capitals',
       'caps_vs_length', 'num_exclamation_marks', 'num_question_marks',
       'num_unique_words', 'words_vs_unique'],
      dtype='object')

<b> Further, we select the best features amongst the list of features on the basis of how they are related to each other. 
So, we consider features such as capital word count, word count, number of exclamation and question marks, number of unique words and unique word ratio as our best features.<b>

In [197]:
# making a copy of the dataframe to ensure re-usability in case of errors
df1 = df.copy()
print('checkpoint')

checkpoint


In [198]:
#feature selection step
#keeping the best features and dropping the rest
drop_list = ['emojis', 'char_count', 'word_density', 'punc_count', 'total_length',
       'caps_vs_length']
df.drop(columns=drop_list, axis = 1, inplace=True)
df.head()

,text,label,num_emojis,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
0,The new rule is - \r\nif you are waiting for a...,4,0,111,2,0,1,86,0.774775
1,"Flirted with giving this two stars, but that's...",3,0,206,0,0,0,144,0.699029
2,I was staying at planet Hollywood across the s...,5,0,111,0,1,0,79,0.711712
3,Food is good but prices are super expensive. ...,2,0,130,0,0,1,98,0.753846
4,Worse company to deal with they do horrible wo...,1,0,124,0,0,0,79,0.637097


We now make a study into the more relevant words by conducting a review into the top bigrams and relevant pos tags.

We then use these to make a cleaned data column which will then be used as tfidf features in our models.

We start by making a document-token pairing of all the current words, and use them to make bigrams. These bigrams are then filtered based on their importance score, which we have taken to be the **Point-wise Mutual Information** scores.

The top 800 bigrams have been taken based on these scores, and those bigrams have been POS tagged.

Following this, we have kept those bigrams which have relevant POS tags, and removed the rest. 

These bigrams were then re-tokenized by the MWETokenizer, and then put into a new dataframe.

In [199]:
# Document-token pairing by making a dictionary where the keys are the doc ids and the values are
# the tokens in the document
tokenized_reuters =  dict(CleanData(row_id,row['text']) for row_id,row in df.iterrows())

In [200]:
# list of all words in the dataset
words = list(chain.from_iterable(tokenized_reuters.values()))

In [201]:
# List of important words which do not appead in the stop words list created earlier
words_imp = [w for w in words if not w in stop_words]

In [202]:
# finding bigrams and trigrams
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(words_imp)


In [203]:
# finding the most meaningful bigrams by using high pmi scores
best_bigrams=bigramFinder.nbest(bigrams.pmi, 800)

In [204]:
# Choose the relevant bigrams which have the POS tags we desire
# Add them to a tagged list
pos_tagged = list()
for bigram in best_bigrams:
    tagged_sentence = nltk.tag.pos_tag([bigram[0].strip('\''),bigram[1].strip('\'')])
    
    if tagged_sentence[0][1] in POS_TAGS or tagged_sentence[1][1] in POS_TAGS:
        pos_tagged.append((tagged_sentence[0][0],tagged_sentence[1][0]))

In [205]:
# Tokenizing the multi-word entities
mwetokens = MWETokenizer(pos_tagged)

In [206]:
# setting a collocations dictionary
colloc_units = dict()
for key, value in tokenized_reuters.items():
    colloc_units[key] = mwetokens.tokenize(value)

In [208]:
# We now make a new dataframe which has the cleaned text, made more relevant by pos tagging and top bigram selection in it's
# text column
# We use the same ids and columns from the original dataframe

ids = [i for i in range(0,df.shape[0])]
df['ids'] = ids
train_df=pd.DataFrame(colloc_units.items(), columns=['ids','text'])
train_df['text']=train_df['text'].astype(str)
train_df['label'] = df['label']
train_df['word_count'] = df['word_count']
train_df['capitals'] = df['capitals']
train_df['num_exclamation_marks'] = df['num_exclamation_marks']
train_df['num_question_marks'] = df['num_question_marks']
train_df['num_unique_words'] = df['num_unique_words']
train_df['words_vs_unique'] = df['words_vs_unique']
train_df.head()

,ids,text,label,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
0,0,"['the', 'new', 'rule', 'is', 'if', 'you', 'are...",4,111,2,0,1,86,0.774775
1,1,"['flirted', 'with', 'giving', 'this', 'two', '...",3,206,0,0,0,144,0.699029
2,2,"['was', 'staying', 'at', 'planet', 'hollywood'...",5,111,0,1,0,79,0.711712
3,3,"['food', 'is', 'good', 'but', 'prices', 'are',...",2,130,0,0,1,98,0.753846
4,4,"['worse', 'company', 'to', 'deal', 'with', 'th...",1,124,0,0,0,79,0.637097


We chose to de-tokenize the tokens before making TF-IDF features. Thus we have merged them so that they form coherent sentences instead of appearing as a list of tokens.

In [209]:
#function to merge the list of tokens into strings
def merge_tokens(x):
    x = re.sub(r'\[|\]','',x)
    x = re.sub(r'[\'\']','',x)
    x = re.sub(r',','',x)
    return x
train_df['text'] = train_df['text'].apply(merge_tokens)

In [210]:
train_df.head()

,ids,text,label,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
0,0,the new rule is if you are waiting for table w...,4,111,2,0,1,86,0.774775
1,1,flirted with giving this two stars but that is...,3,206,0,0,0,144,0.699029
2,2,was staying at planet hollywood across the str...,5,111,0,1,0,79,0.711712
3,3,food is good but prices are super expensive NU...,2,130,0,0,1,98,0.753846
4,4,worse company to deal with they do horrible wo...,1,124,0,0,0,79,0.637097


In [211]:
# max_features is an upper bound on the number of words in the vocabulary that will be made once the vectorizer runs
max_features = 40000

In [212]:
# train test split (80:20 ratio)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df.drop(columns=['label','ids'], axis=1), df.label, \
                                                    test_size=0.2, random_state=42)

In [213]:
# Taking a look at our training split
X_train

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
39087,have never been so out done lucille is we were...,194,0,2,0,125,0.644330
30893,this is my go to spot always try something new...,35,0,1,0,27,0.771429
45278,would say meh but all in all it is not terribl...,214,2,0,0,138,0.644860
16398,this place has large variety of drinks and sna...,121,0,0,0,89,0.735537
13653,horrid own business and do deposits daily have...,77,3,4,0,61,0.792208
...,...,...,...,...,...,...,...
11284,this is hidden gem in king west this is my go ...,63,0,3,0,49,0.777778
44732,am vegas clubbing veteran would rather not dis...,279,3,1,0,185,0.663082
38158,great staff great vet had great experience wit...,21,0,1,0,19,0.904762
860,have loved this restaurant since the first tim...,72,0,1,0,56,0.777778


In [214]:
# taking a look at our training split labels
y_train

39087    1
30893    5
45278    3
16398    3
13653    2
        ..
11284    5
44732    5
38158    5
860      5
15795    2
Name: label, Length: 40000, dtype: int64

In [215]:
# we take the numerical columns into this new list
textcountscols = X_train.columns[1:]

In [ ]:
# Since we have numerical features, we need to normalise them so that there is to ambiguity in their scales
# Furthermore after the creation of the new dataframe, some columns have a datatype of "Object",
# which needs to be converted to numeric

# 1. Conversion of datatype
# 2. Min-Max Feature Scaling to scale the features
for colname in textcountscols:
    X_train[colname] = pd.to_numeric(X_train[colname])
    X_train[colname] = (X_train[colname]-X_train[colname].min())/(X_train[colname].max()-X_train[colname].min())
X_train.dtypes

In [217]:
for colname in textcountscols:
    X_test[colname] = pd.to_numeric(X_test[colname])
    X_test[colname] = (X_test[colname]-X_test[colname].min())/(X_test[colname].max()-X_test[colname].min())
X_test.dtypes

text                      object
word_count               float64
capitals                 float64
num_exclamation_marks    float64
num_question_marks       float64
num_unique_words         float64
words_vs_unique          float64
dtype: object

In [218]:
# Class for custom feature extraction pipeline
from sklearn.base import BaseEstimator, TransformerMixin
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols
    def transform(self, X, **transform_params):
        return X[self.cols]
    def fit(self, X, y=None, **fit_params):
        return self

## Model Development

We have considered 3 Models for our classification task:

1. Logisitc Regression
2. Multinomial Naive Bayes
3. Ensembled model of Logistic and Naive Bayes, and a Stacked model which consists of the ensembled model, and the base logistic and naive bayes models. 

**NOTE** Only the logistic model gives the highest accuracy so we have removed the code for the implementation of the other models.


###  Logistic Regression Model

In [220]:
# Building the logistic regression model by integrating custom features using FeatureUnion 
# and vectorizing those features using TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion

tfidf_vect= TfidfVectorizer(use_idf=True, sublinear_tf = True, max_features = max_features, lowercase=False,\
                            token_pattern=r'\w{2,}', ngram_range=(1,3), encoding='utf-8')

features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols)),\
                           ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='text')), ('vect', tfidf_vect)]))])
log_model = Pipeline([('features', features),
                        ('log', OneVsRestClassifier(log_r(random_state=0, C=1.01, solver='lbfgs')))])


In [221]:
#model fitting
log_model.fit(X_train.fillna(' '), y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('textcounts',
                                                 ColumnExtractor(cols=Index(['word_count', 'capitals', 'num_exclamation_marks', 'num_question_marks',
       'num_unique_words', 'words_vs_unique'],
      dtype='object'))),
                                                ('pipe',
                                                 Pipeline(memory=None,
                                                          steps=[('cleantext',
                                                                  ColumnExtractor(cols='text')),
                                                                 ('vect',
                                                                  TfidfVectoriz...
                                                          verbose=False))],
                              transformer_weights=None, verbose=False)),
           

In [222]:
#reporting the accuracy of the model
y_log_pred = log_model.predict(X_test.fillna(' '))
print("Accuracy:",accuracy_score(y_test, y_log_pred))

Accuracy: 0.61


<b> The accuracy shown by the logistic regression model is 61% on the labelled dataset.<b>

<b> The accuracy shown by the ensemble model is 60.07% on the labelled dataset <b>

<b> The accuracy shown by the stacking model is 60.24% on the labelled dataset <b>

<b> Out of the above 3 models, we have chosen the best model as Logistic Regression since it gives an high accuracy. </b> <br>
We further use that model to predict labels on the unlabelled dataset as shown in the steps below.

First, we pass the unlabelled dataset to the preprocessing pipeline in a similar fashion as we did it for the labelled dataset.

## Pre-processing unlabeled data and feature extraction

## We have followed the same pre-processing and feature extraction steps as above.

In [231]:
#reading the unlabeled dataset
unlabeled=pd.read_csv('unlabeled_data.csv')
unlabeled.head()

,text
0,Had a good experience when my wife and I sat a...
1,On my first to Montreal with my gf we came her...
2,One of our favorite places to go when it's col...
3,"The doctor was very nice, got in in a good amo..."
4,The Nook is an immediate phoenix staple! I ca...


Through some visual analysis, we see that labeled dataset is a subset of unlabeled dataset. So, we drop those rows from the unlabeled dataset which exist in the labeled dataset by performing an outer join on both the datasets as shown below.

In [233]:
#merging two datasets using outerjoin
new_df=pd.merge(df, unlabeled, on='text', how='outer')
new_df.head(2)

,text,label,num_emojis,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique,ids
0,The new rule is - \r\nif you are waiting for a...,4.0,0.0,111.0,2.0,0.0,1.0,86.0,0.774775,0.0
1,"Flirted with giving this two stars, but that's...",3.0,0.0,206.0,0.0,0.0,0.0,144.0,0.699029,1.0


In [234]:
#length of new dataframe
len(new_df)

605123

In [235]:
#checking for NaN values
new_df.isnull().sum()

text                          0
label                    554909
num_emojis               554909
word_count               554909
capitals                 554909
num_exclamation_marks    554909
num_question_marks       554909
num_unique_words         554909
words_vs_unique          554909
ids                      554909
dtype: int64

We further see that the outer join creates NaN values in the quantitative features columns and label columns. We want those rows which have NaN values as a part of our final unlabeled dataset. In this way, duplicate rows have been removed from the unlabeled dataset.

In [236]:
#final unlabeled dataset
unlabeled = new_df[new_df['label'].isnull()]
unlabeled.head()

,text,label,num_emojis,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique,ids
50214,Had a good experience when my wife and I sat a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50215,On my first to Montreal with my gf we came her...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50216,One of our favorite places to go when it's col...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50217,"The doctor was very nice, got in in a good amo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50218,The Nook is an immediate phoenix staple! I ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, we remove those columns having NaN and use this updated unlabeled dataset for preprocessing and feature extraction

In [237]:
unlabeled.drop(columns=['label','num_emojis','word_count','capitals','num_exclamation_marks','num_question_marks','num_unique_words','words_vs_unique','ids'], axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [238]:
unlabeled.head(2)

,text
50214,Had a good experience when my wife and I sat a...
50215,On my first to Montreal with my gf we came her...


In [239]:
len(unlabeled)

554909

Further, we pass the unlabeled data to the feature extraction function where we generate some quantitative features for the textual data.

In [240]:
#extracting features
unlabeled=extract_features(unlabeled)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

Done


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


We preserve the best features and drop the irrelevant features.

In [241]:
#keeping the  best features
drop_list = ['char_count', 'word_density', 'punc_count', 'total_length',
       'caps_vs_length']
unlabeled.drop(columns=drop_list, axis = 1, inplace=True)
unlabeled.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
50214,Had a good experience when my wife and I sat a...,90,0,0,0,73,0.811111
50215,On my first to Montreal with my gf we came her...,296,0,0,0,193,0.652027
50216,One of our favorite places to go when it's col...,27,0,1,0,26,0.962963
50217,"The doctor was very nice, got in in a good amo...",140,2,3,0,97,0.692857
50218,The Nook is an immediate phoenix staple! I ca...,197,0,1,0,126,0.639594


Now, we pass the textual data to the preprocessing function where the raw reviews are being cleaned. We further remove the stopwords from the entire corpus of reviews and generate best bigrams on the basis of pointwise mutual information (pmi) score.

In [242]:
# Document token pairing by making a dictionary where the keys are the doc ids and the values are
# the tokens in the document
tokenized_reuters =  dict(CleanData(row_id,row['text']) for row_id,row in unlabeled.iterrows())


# list of all words in the dataset
words = list(chain.from_iterable(tokenized_reuters.values()))

# list of words not in stop words
words_imp = [w for w in words if not w in stop_words]

In [243]:
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(words_imp)

In [244]:
# finding the most meaningful bigrams by using high pmi scores
best_bigrams=bigramFinder.nbest(bigrams.pmi, 800)

In [245]:
# Choose the relevant bigrams which have the POS tags we desire
# Add them to a tagged list
pos_tagged = list()
for bigram in best_bigrams:
    tagged_sentence = nltk.tag.pos_tag([bigram[0].strip('\''),bigram[1].strip('\'')])
    
    if tagged_sentence[0][1] in POS_TAGS or tagged_sentence[1][1] in POS_TAGS:
        pos_tagged.append((tagged_sentence[0][0],tagged_sentence[1][0]))

Further, we tokenize the multi-word bigrams using MWETokenizer as shown below.

In [246]:
# Tokenizing the multi-word entities
mwetokens = MWETokenizer(pos_tagged)

colloc_units = dict()
for key, value in tokenized_reuters.items():
    colloc_units[key] = mwetokens.tokenize(value)

In [247]:
ids = [i for i in range(0,unlabeled.shape[0])]
unlabeled['ids'] = ids

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [248]:
#extracting data from dictionary into a dataframe
train_df_unlab=pd.DataFrame(colloc_units.items(), columns=['ids','text'])
train_df_unlab['text']=train_df_unlab['text'].astype(str)

In [249]:
train_df_unlab.head(2)

,ids,text
0,50214,"['had', 'good', 'experience', 'when', 'my', 'w..."
1,50215,"['on', 'my', 'first', 'to', 'montreal', 'with'..."


In [250]:
train_df_unlab.set_index('ids', inplace=True)

In [251]:
#appending the quantitative features from unlabeled dataset to new dataset
train_df_unlab['word_count'] = unlabeled['word_count']

In [252]:
train_df_unlab['capitals'] = unlabeled['capitals']
train_df_unlab['num_exclamation_marks'] = unlabeled['num_exclamation_marks']
train_df_unlab['num_question_marks'] = unlabeled['num_question_marks']
train_df_unlab['num_unique_words'] = unlabeled['num_unique_words']
train_df_unlab['words_vs_unique'] = unlabeled['words_vs_unique']
train_df_unlab.head()

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
ids,,,,,,,
50214,"['had', 'good', 'experience', 'when', 'my', 'w...",90,0,0,0,73,0.811111
50215,"['on', 'my', 'first', 'to', 'montreal', 'with'...",296,0,0,0,193,0.652027
50216,"['one', 'of', 'our', 'favorite', 'places', 'to...",27,0,1,0,26,0.962963
50217,"['the', 'doctor', 'was', 'very', 'nice', 'got'...",140,2,3,0,97,0.692857
50218,"['the', 'nook', 'is', 'an', 'immediate', 'phoe...",197,0,1,0,126,0.639594


In [253]:
#merging the tokens into strings
train_df_unlab['text'] = train_df_unlab['text'].apply(merge_tokens)

In [254]:
train_df_unlab.head()

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
ids,,,,,,,
50214,had good experience when my wife and sat at th...,90,0,0,0,73,0.811111
50215,on my first to montreal with my gf we came her...,296,0,0,0,193,0.652027
50216,one of our favorite places to go when it is co...,27,0,1,0,26,0.962963
50217,the doctor was very nice got in in good amount...,140,2,3,0,97,0.692857
50218,the nook is an immediate phoenix staple came h...,197,0,1,0,126,0.639594


In [255]:
#Min-Max Normalization of the extracted quantitative features.
for colname in textcountscols:
    train_df_unlab[colname] = pd.to_numeric(train_df_unlab[colname])
    train_df_unlab[colname] = (train_df_unlab[colname]-train_df_unlab[colname].min())/(train_df_unlab[colname].max()-\
                                                                                          train_df_unlab[colname].min())
train_df_unlab.dtypes

text                      object
word_count               float64
capitals                 float64
num_exclamation_marks    float64
num_question_marks       float64
num_unique_words         float64
words_vs_unique          float64
dtype: object

Now, we use the logistic model to predict the labels on the unlabelled dataset as shown below.

In [256]:
# Get the class probabilites
# predict_proba gives a vector of values which correspond to the probability of that observation belonging to each class
y_train_unlab_proba = log_model.predict_proba(train_df_unlab.fillna(' '))

In [257]:
#appending the probabilities to a dataframe
y_train_unlab_proba = list(y_train_unlab_proba)
y_train_unlab_proba

[array([0.30296795, 0.13528451, 0.28217507, 0.18408618, 0.0954863 ]),
 array([0.00619308, 0.12128443, 0.33860192, 0.36274745, 0.17117312]),
 array([0.03860827, 0.08587971, 0.08760938, 0.16113237, 0.62677026]),
 array([0.58803309, 0.17142658, 0.0997028 , 0.06039315, 0.08044438]),
 array([0.0073796 , 0.01551592, 0.12903339, 0.14341813, 0.70465297]),
 array([0.25572927, 0.20696401, 0.07486961, 0.20477561, 0.25766151]),
 array([0.02791756, 0.08525593, 0.08121778, 0.43761734, 0.36799139]),
 array([0.1794823 , 0.59406279, 0.19623708, 0.02262795, 0.00758988]),
 array([0.01160255, 0.03508851, 0.4207284 , 0.37811107, 0.15446948]),
 array([0.01453531, 0.1920774 , 0.32295283, 0.34370752, 0.12672694]),
 array([0.25467369, 0.34815149, 0.18434181, 0.07449309, 0.13833992]),
 array([0.1273478 , 0.14169209, 0.06688821, 0.10035799, 0.56371391]),
 array([0.14260298, 0.27113736, 0.33286977, 0.19499746, 0.05839243]),
 array([0.00360046, 0.00529366, 0.00965459, 0.26627194, 0.71517935]),
 array([0.0244877 , 

In [258]:
# Get the class labels
# predict method returns the class label of the observation based on the highest class probability 
y_train_unlab =  log_model.predict(train_df_unlab.fillna(' '))

In [259]:
#appending the predictions to a dataframe
y_train_unlab = list(y_train_unlab)
y_train_unlab

[1,
 4,
 5,
 1,
 5,
 5,
 4,
 2,
 3,
 4,
 2,
 5,
 3,
 5,
 5,
 4,
 1,
 1,
 3,
 4,
 4,
 4,
 1,
 2,
 3,
 4,
 2,
 4,
 4,
 5,
 4,
 1,
 5,
 5,
 3,
 5,
 1,
 2,
 3,
 5,
 4,
 4,
 3,
 3,
 1,
 4,
 4,
 3,
 1,
 1,
 4,
 3,
 2,
 4,
 4,
 4,
 4,
 3,
 3,
 1,
 4,
 1,
 1,
 2,
 4,
 2,
 3,
 5,
 4,
 3,
 4,
 3,
 3,
 4,
 1,
 5,
 5,
 2,
 2,
 1,
 3,
 5,
 1,
 4,
 1,
 4,
 3,
 5,
 5,
 4,
 3,
 5,
 5,
 1,
 5,
 4,
 4,
 1,
 1,
 3,
 1,
 5,
 4,
 4,
 1,
 4,
 1,
 4,
 2,
 2,
 3,
 1,
 5,
 3,
 2,
 5,
 2,
 2,
 1,
 3,
 3,
 4,
 5,
 4,
 2,
 1,
 3,
 5,
 1,
 2,
 3,
 3,
 5,
 4,
 4,
 3,
 1,
 5,
 2,
 5,
 1,
 3,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 3,
 1,
 5,
 2,
 3,
 3,
 1,
 2,
 4,
 2,
 4,
 5,
 1,
 3,
 4,
 4,
 5,
 2,
 2,
 1,
 4,
 5,
 4,
 5,
 1,
 2,
 3,
 3,
 3,
 4,
 2,
 5,
 1,
 5,
 5,
 1,
 1,
 4,
 2,
 3,
 4,
 2,
 4,
 2,
 5,
 4,
 3,
 5,
 5,
 3,
 5,
 1,
 2,
 1,
 3,
 3,
 4,
 1,
 4,
 5,
 5,
 5,
 5,
 1,
 4,
 5,
 5,
 1,
 1,
 4,
 2,
 3,
 5,
 2,
 1,
 2,
 5,
 5,
 2,
 1,
 3,
 1,
 3,
 1,
 1,
 3,
 1,
 3,
 2,
 4,
 5,
 2,
 4,
 5,


In [260]:
train_df_unlab.head()

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
ids,,,,,,,
50214,had good experience when my wife and sat at th...,0.085577,0.000000,0.000000,0.0,0.130672,0.763889
50215,on my first to montreal with my gf we came her...,0.283654,0.000000,0.000000,0.0,0.348457,0.565034
50216,one of our favorite places to go when it is co...,0.025000,0.000000,0.006623,0.0,0.045372,0.953704
50217,the doctor was very nice got in in good amount...,0.133654,0.009174,0.019868,0.0,0.174229,0.616071
50218,the nook is an immediate phoenix staple came h...,0.188462,0.000000,0.006623,0.0,0.226860,0.549492


In [261]:
# setting the label and the probability columns in our dataframe
train_df_unlab['label'] = y_train_unlab
train_df_unlab['proba'] = y_train_unlab_proba
train_df_unlab.head()

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique,label,proba
ids,,,,,,,,,
50214,had good experience when my wife and sat at th...,0.085577,0.000000,0.000000,0.0,0.130672,0.763889,1,"[0.3029679508072741, 0.13528450560284286, 0.28..."
50215,on my first to montreal with my gf we came her...,0.283654,0.000000,0.000000,0.0,0.348457,0.565034,4,"[0.006193083729534292, 0.12128443188432902, 0...."
50216,one of our favorite places to go when it is co...,0.025000,0.000000,0.006623,0.0,0.045372,0.953704,5,"[0.03860827433542501, 0.08587971408413816, 0.0..."
50217,the doctor was very nice got in in good amount...,0.133654,0.009174,0.019868,0.0,0.174229,0.616071,1,"[0.5880330922926519, 0.17142657793184435, 0.09..."
50218,the nook is an immediate phoenix staple came h...,0.188462,0.000000,0.006623,0.0,0.226860,0.549492,5,"[0.007379595186535904, 0.015515921839902561, 0..."


**Now, logistic regression model gives a probability values for each observation. 
We select those observations which have class probability values of more than 0.8. 
Doing so we ensure that we only add those pseudo labels for which the model is atleast 80% confident.
We add those pseudo labelled rows from our unlabeled dataset to the labeled dataset as shown below.**

In [262]:
max_proba_list = []

for rowlist in train_df_unlab['proba']:
    max_proba_list.append(max(rowlist))
    
train_df_unlab['max_proba'] = max_proba_list
train_df_unlab.head()

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique,label,proba,max_proba
ids,,,,,,,,,,
50214,had good experience when my wife and sat at th...,0.085577,0.000000,0.000000,0.0,0.130672,0.763889,1,"[0.3029679508072741, 0.13528450560284286, 0.28...",0.302968
50215,on my first to montreal with my gf we came her...,0.283654,0.000000,0.000000,0.0,0.348457,0.565034,4,"[0.006193083729534292, 0.12128443188432902, 0....",0.362747
50216,one of our favorite places to go when it is co...,0.025000,0.000000,0.006623,0.0,0.045372,0.953704,5,"[0.03860827433542501, 0.08587971408413816, 0.0...",0.626770
50217,the doctor was very nice got in in good amount...,0.133654,0.009174,0.019868,0.0,0.174229,0.616071,1,"[0.5880330922926519, 0.17142657793184435, 0.09...",0.588033
50218,the nook is an immediate phoenix staple came h...,0.188462,0.000000,0.006623,0.0,0.226860,0.549492,5,"[0.007379595186535904, 0.015515921839902561, 0...",0.704653


In [263]:
#taking rows having probability values more than 0.80 (high confidence rows) 
#This threshold indicates how certain the model is
train_df_unlab = train_df_unlab[train_df_unlab.max_proba > 0.80]
train_df_unlab

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique,label,proba,max_proba
ids,,,,,,,,,,
50293,do not get your policy with them worst custome...,0.070192,0.032110,0.000000,0.000000,0.114338,0.831081,1,"[0.8599410810652746, 0.0880713238120241, 0.035...",0.859941
50326,just love this place everyone there is so nice...,0.037500,0.000000,0.013245,0.000000,0.052632,0.687500,5,"[0.007364156786535241, 0.018323603818547887, 0...",0.813797
50354,all problems were small to negligible until go...,0.116346,0.009174,0.019868,0.000000,0.156080,0.641393,1,"[0.8526026406729331, 0.07957033129108847, 0.03...",0.852603
50389,have used this company for few of my propertie...,0.144231,0.004587,0.006623,0.000000,0.183303,0.594371,1,"[0.8147498939757117, 0.06730999309230402, 0.05...",0.814750
50418,absolutely amazing food even better service th...,0.015385,0.004587,0.006623,0.000000,0.027223,0.926471,5,"[0.022481002779077424, 0.0153826785039694, 0.0...",0.908612
...,...,...,...,...,...,...,...,...,...,...
605056,this is by far the worst company my family and...,0.275000,0.022936,0.006623,0.000000,0.317604,0.516551,1,"[0.8112203153354156, 0.0812188282745403, 0.028...",0.811220
605061,would not even give them NUM star this is the ...,0.090385,0.000000,0.013245,0.000000,0.139746,0.776316,1,"[0.870414921041628, 0.036086582936979125, 0.02...",0.870415
605073,if could give no stars would about week ago sc...,0.250000,0.045872,0.006623,0.015385,0.284936,0.506705,1,"[0.8358205461496548, 0.1090832884571438, 0.033...",0.835821


In [264]:
#dropping maximum probability column and probability vector column
train_df_unlab = train_df_unlab.drop(columns=['max_proba', 'proba'], axis=1)
train_df_unlab.head()

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique,label
ids,,,,,,,,
50293,do not get your policy with them worst custome...,0.070192,0.032110,0.000000,0.0,0.114338,0.831081,1
50326,just love this place everyone there is so nice...,0.037500,0.000000,0.013245,0.0,0.052632,0.687500,5
50354,all problems were small to negligible until go...,0.116346,0.009174,0.019868,0.0,0.156080,0.641393,1
50389,have used this company for few of my propertie...,0.144231,0.004587,0.006623,0.0,0.183303,0.594371,1
50418,absolutely amazing food even better service th...,0.015385,0.004587,0.006623,0.0,0.027223,0.926471,5


In [265]:
#Min-Max Normalization 
for colname in textcountscols:
    train_df[colname] = pd.to_numeric(train_df[colname])
    train_df[colname] = (train_df[colname]-train_df[colname].min())/(train_df[colname].max()-\
                                                                           train_df[colname].min())
train_df.dtypes

ids                        int64
text                      object
label                      int64
word_count               float64
capitals                 float64
num_exclamation_marks    float64
num_question_marks       float64
num_unique_words         float64
words_vs_unique          float64
dtype: object

In [266]:
train_df_unlab.reset_index('ids', inplace=True)

In [267]:
train_df_unlab= train_df_unlab[['ids','text','label','word_count','capitals','num_exclamation_marks','num_question_marks','num_unique_words','words_vs_unique']]
train_df_unlab.head()

,ids,text,label,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
0,50293,do not get your policy with them worst custome...,1,0.070192,0.032110,0.000000,0.0,0.114338,0.831081
1,50326,just love this place everyone there is so nice...,5,0.037500,0.000000,0.013245,0.0,0.052632,0.687500
2,50354,all problems were small to negligible until go...,1,0.116346,0.009174,0.019868,0.0,0.156080,0.641393
3,50389,have used this company for few of my propertie...,1,0.144231,0.004587,0.006623,0.0,0.183303,0.594371
4,50418,absolutely amazing food even better service th...,5,0.015385,0.004587,0.006623,0.0,0.027223,0.926471


In [268]:
#merging both the dataframes
merged_df = pd.concat([train_df,train_df_unlab], ignore_index = True)

In [269]:
merged_df = merged_df.drop(columns=['ids'],axis = 1)
merged_df.head()

,text,label,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
0,the new rule is if you are waiting for table w...,4,0.107738,0.02,0.000000,0.027027,0.158582,0.643544
1,flirted with giving this two stars but that is...,3,0.200784,0.00,0.000000,0.000000,0.266791,0.523664
2,was staying at planet hollywood across the str...,5,0.107738,0.00,0.011765,0.000000,0.145522,0.543736
3,food is good but prices are super expensive NU...,2,0.126347,0.00,0.000000,0.027027,0.180970,0.610421
4,worse company to deal with they do horrible wo...,1,0.120470,0.00,0.000000,0.000000,0.145522,0.425646


In [270]:
merged_df.shape

(67519, 8)

In this manner, the high confidence rows from unlabeled dataset has been added to the labeled dataset and now we rebuild the logistic regression, multinomial naive bayes and ensemble models.

In [271]:
#train test split in the ratio of 80:20
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(merged_df.drop('label', axis=1), merged_df.label, \
                                                    test_size=0.2, random_state=69)

In [272]:
X_train

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
23486,stayed at the rio this weekend and will be my ...,0.324192,0.020000,0.000000,0.000000,0.363806,0.351680
33911,exactly what needed when wanted quick snack lo...,0.042116,0.000000,0.000000,0.000000,0.072761,0.856121
41958,have been here couple times now and have had m...,0.064643,0.000000,0.011765,0.000000,0.100746,0.716538
10319,ate there once and short of being annoyed by a...,0.068560,0.000000,0.000000,0.000000,0.110075,0.754798
21177,ate here last week initially enjoyed the ambia...,0.273262,0.000000,0.000000,0.000000,0.296642,0.321715
...,...,...,...,...,...,...,...
11825,we loved our stay at the venetian wish we coul...,0.358472,0.020000,0.176471,0.054054,0.492537,0.560131
47508,not bad but have had better shumai their taro ...,0.024486,0.000000,0.000000,0.000000,0.046642,1.000000
25015,had happy hour for the day after my wedding wh...,0.058766,0.000000,0.000000,0.000000,0.093284,0.740547
9818,this is the only place that know of that can g...,0.123408,0.010000,0.000000,0.000000,0.173507,0.588756


In [273]:
y_train

23486    1
33911    4
41958    4
10319    2
21177    1
        ..
11825    5
47508    3
25015    5
9818     3
64566    1
Name: label, Length: 54015, dtype: int64

# Running logistic model

In [274]:
#running logistic model on pseudo labels + train labels
log_model.fit(X_train.fillna(' '), y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('textcounts',
                                                 ColumnExtractor(cols=Index(['word_count', 'capitals', 'num_exclamation_marks', 'num_question_marks',
       'num_unique_words', 'words_vs_unique'],
      dtype='object'))),
                                                ('pipe',
                                                 Pipeline(memory=None,
                                                          steps=[('cleantext',
                                                                  ColumnExtractor(cols='text')),
                                                                 ('vect',
                                                                  TfidfVectoriz...
                                                          verbose=False))],
                              transformer_weights=None, verbose=False)),
           

In [275]:
y_log_pred = log_model.predict(X_test.fillna(' '))
print("Accuracy:",accuracy_score(y_test, y_log_pred))

Accuracy: 0.7133441943127962


**The accuracy shown by logistic regression model on labelled+unlabelled dataset is 71.33**

**Accuracy shown by Multinomial Naive Bayes was 68.60 on labelled+unlabelled dataset**

**The accuracy shown by Ensemble Model on labelled+unlabelled dataset was 70.93**

**The accuracy shown by Stacked Model on labelled+unlabelled dataset was 71.05**

## Preprocessing test data and feature extraction

Now, we perform predictions on the test data using logistic regression model and evaluate its accuracy.

In [304]:
#reading the test file
kaggle=pd.read_csv('test_data.csv')
kaggle.head()

,test_id,text
0,test_1,trying to have a nice quiet dinner. the annou...
1,test_2,Been getting food to go from here for over 3yr...
2,test_3,Ugh. I've had to eat here a couple of times be...
3,test_4,The people here are so nice! I ordered on eat ...
4,test_5,Heard alot of good things about this place and...


We pass the test dataset to the preprocessing function and we also generate numerical features as we did for the labelled dataset.

In [305]:
#generating numerical features
kaggle=extract_features(kaggle)

Done


In [306]:
# Document token pairing by making a dictionary where the keys are the doc ids and the values are
# the tokens in the document
tokenized_reuters =  dict(CleanData(row_id,row['text']) for row_id,row in kaggle.iterrows())


# list of all words in the dataset
words = list(chain.from_iterable(tokenized_reuters.values()))

# list of words not in stop words
words_imp = [w for w in words if not w in stop_words]

In [307]:
#keeping the best features
drop_list = ['char_count', 'word_density', 'punc_count', 'total_length',
       'caps_vs_length']
kaggle.drop(columns=drop_list, axis = 1, inplace=True)
kaggle.head()

,test_id,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
0,test_1,trying to have a nice quiet dinner. the annou...,20,0,0,0,18,0.900000
1,test_2,Been getting food to go from here for over 3yr...,124,0,0,0,98,0.790323
2,test_3,Ugh. I've had to eat here a couple of times be...,248,7,3,0,172,0.693548
3,test_4,The people here are so nice! I ordered on eat ...,32,0,3,0,31,0.968750
4,test_5,Heard alot of good things about this place and...,95,0,2,0,79,0.831579


In [308]:
#processing bigrams
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(words_imp)

In [309]:
best_bigrams=bigramFinder.nbest(bigrams.pmi, 200)

In [310]:
# Choose the relevant bigrams which have the POS tags we desire
# Add them to a tagged list
pos_tagged = list()
for bigram in best_bigrams:
    tagged_sentence = nltk.tag.pos_tag([bigram[0].strip('\''),bigram[1].strip('\'')])
    
    if tagged_sentence[0][1] in POS_TAGS or tagged_sentence[1][1] in POS_TAGS:
        pos_tagged.append((tagged_sentence[0][0],tagged_sentence[1][0]))

In [311]:
# Tokenizing the multi-word entities
mwetokens = MWETokenizer(pos_tagged)

colloc_units = dict()
for key, value in tokenized_reuters.items():
    colloc_units[key] = mwetokens.tokenize(value)

In [312]:
#appending numerical features
ids = [i for i in range(0,kaggle.shape[0])]
kaggle['ids'] = ids
train_df_kaggle=pd.DataFrame(colloc_units.items(), columns=['ids','text'])
train_df_kaggle['text']=train_df_kaggle['text'].astype(str)
train_df_kaggle['word_count'] = kaggle['word_count']
train_df_kaggle['capitals'] = kaggle['capitals']
train_df_kaggle['num_exclamation_marks'] = kaggle['num_exclamation_marks']
train_df_kaggle['num_question_marks'] = kaggle['num_question_marks']
train_df_kaggle['num_unique_words'] = kaggle['num_unique_words']
train_df_kaggle['words_vs_unique'] = kaggle['words_vs_unique']
train_df_kaggle.head()

,ids,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
0,0,"['trying', 'to', 'have', 'nice', 'quiet', 'din...",20,0,0,0,18,0.900000
1,1,"['been', 'getting', 'food', 'to', 'go', 'from'...",124,0,0,0,98,0.790323
2,2,"['ugh', 'have', 'had', 'to', 'eat', 'here', 'c...",248,7,3,0,172,0.693548
3,3,"['the', 'people', 'here', 'are', 'so', 'nice',...",32,0,3,0,31,0.968750
4,4,"['heard', 'alot', 'of', 'good', 'things', 'abo...",95,0,2,0,79,0.831579


In [313]:
train_df_kaggle.drop('ids',axis=1,inplace=True)
train_df_kaggle.head()

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
0,"['trying', 'to', 'have', 'nice', 'quiet', 'din...",20,0,0,0,18,0.900000
1,"['been', 'getting', 'food', 'to', 'go', 'from'...",124,0,0,0,98,0.790323
2,"['ugh', 'have', 'had', 'to', 'eat', 'here', 'c...",248,7,3,0,172,0.693548
3,"['the', 'people', 'here', 'are', 'so', 'nice',...",32,0,3,0,31,0.968750
4,"['heard', 'alot', 'of', 'good', 'things', 'abo...",95,0,2,0,79,0.831579


In [314]:
#merging the tokens
train_df_kaggle['text'] = train_df_kaggle['text'].apply(merge_tokens)

In [315]:
#Min-Max Normalization
for colname in textcountscols:
    train_df_kaggle[colname] = pd.to_numeric(train_df_kaggle[colname])
    train_df_kaggle[colname] = (train_df_kaggle[colname]-train_df_kaggle[colname].min())/(train_df_kaggle[colname].max()-\
                                                                                          train_df_kaggle[colname].min())
train_df_kaggle.dtypes

text                      object
word_count               float64
capitals                 float64
num_exclamation_marks    float64
num_question_marks       float64
num_unique_words         float64
words_vs_unique          float64
dtype: object

In [316]:
train_df_kaggle

,text,word_count,capitals,num_exclamation_marks,num_question_marks,num_unique_words,words_vs_unique
0,trying to have nice quiet dinner the announcer...,0.018627,0.000000,0.000000,0.000000,0.032015,0.851166
1,been getting food to go from here for over NUM...,0.120588,0.000000,0.000000,0.000000,0.182674,0.687929
2,ugh have had to eat here couple of times becau...,0.242157,0.078652,0.023256,0.000000,0.322034,0.543896
3,the people here are so nice ordered on eat NUM...,0.030392,0.000000,0.023256,0.000000,0.056497,0.953489
4,heard alot of good things about this place and...,0.092157,0.000000,0.015504,0.000000,0.146893,0.749333
...,...,...,...,...,...,...,...
49995,their pizza is alright still think spinato is ...,0.020588,0.000000,0.007752,0.000000,0.033898,0.797045
49996,are here for an event and it was as expected f...,0.020588,0.000000,0.000000,0.000000,0.035782,0.864697
49997,came here for business lunch and was not impre...,0.068627,0.011236,0.000000,0.000000,0.096045,0.601712
49998,have been working with charles over the last N...,0.047059,0.000000,0.000000,0.000000,0.075330,0.757006


In [317]:
#performing predictions
kaggle_log_pred = log_model.predict(train_df_kaggle.fillna(' '))

In [318]:
kaggle['label'] = kaggle_log_pred

In [319]:
kaggle.drop(columns=['text','word_count','capitals','num_exclamation_marks','num_question_marks','num_unique_words','words_vs_unique','ids'], axis=1, inplace=True)

In [320]:
kaggle.shape

(50000, 2)

In [321]:
kaggle.to_csv('predict_label.csv', index=False)

## References


* https://towardsdatascience.com/the-real-world-as-seen-on-twitter-sentiment-analysis-part-one-5ac2d06b63fb
* https://towardsdatascience.com/sentiment-analysis-simplified-ac30720a5827
* https://medium.com/@robert.salgado/multiclass-text-classification-from-start-to-finish-f616a8642538
* https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/
* https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f
* https://towardsdatascience.com/machine-learning-multiclass-classification-with-imbalanced-data-set-29f6a177c1a
* https://www.analyticsvidhya.com/blog/2017/09/pseudo-labelling-semi-supervised-learning-technique/